<a href="https://colab.research.google.com/github/moustafa-7/Contextual-Bandint_RL_Agent/blob/master/Contextual_Bandit_Network_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [0]:
class contextual_bandit():
  def __init__(self):
    self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
    self.num_bandits = self.bandits.shape[0]
    self.num_actions = self.bandits.shape[1]
    
    
  def get_bandit(self):
    self.state = np.random.randint(0,len(self.bandits))
    return self.state
  
  def pull_arm(self,action):
    result = np.random.randn(1)
    bandit = self.bandits[self.state, action]
    
    if result > bandit:
      return 1
    
    else:
      return -1    

## Now build our Policy-Based Agent

In [0]:
class agent():
  def __init__(self, lr, s_size, a_size):
    self.state_in = tf.placeholder(shape =[1], dtype = tf.int32)
    self.state_in_oh = slim.one_hot_encoding(self.state_in, s_size)
    
    output = slim.fully_connected(self.state_in_oh, a_size, activation_fn = tf.nn.sigmoid,
                                 weights_initializer = tf.ones_initializer(),
                                 biases_initializer = None)
    
    self.output = tf.reshape(output,[-1])
    self.chosen_action = tf.argmax(self.output, 0)
    
    
    self.reward_holder = tf.placeholder(shape = [1], dtype = tf.float32)
    self.action_holder = tf.placeholder(shape = [1], dtype = tf.int32)
    
    self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
    self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
    
    self.update = optimizer.minimize(self.loss)
   

## Training the Agent

In [33]:
tf.reset_default_graph()

c_bandit = contextual_bandit()
my_agent = agent(0.001, s_size = c_bandit.num_bandits, a_size = c_bandit.num_actions)

weights = tf.trainable_variables()[0]

num_episodes = 10000

y = 0.99
e = 0.1

total_reward = np.zeros([c_bandit.num_bandits, c_bandit.num_actions])


init = tf.initialize_all_variables()
with tf.Session() as sess:
  sess.run(init)
  
  for i in range(num_episodes):
    s = c_bandit.get_bandit()
    
    if np.random.randn(1) < e:
      a = np.random.randint(c_bandit.num_actions) 
      
    else:
      a = sess.run(my_agent.chosen_action, feed_dict = {my_agent.state_in:[s]})
      
      
    reward = c_bandit.pull_arm(a)
    
    feed_dict = {my_agent.action_holder: [a], my_agent.reward_holder: [reward], my_agent.state_in: [s]}
    _, ww = sess.run([my_agent.update, weights], feed_dict = feed_dict)
    
    
    
    total_reward[s, a] += reward
    
    if(i % 500 == 0):
      print("Mean reward for each of the " + str(c_bandit.num_bandits) + " bandits: " +
           str(np.mean(total_reward, axis = 1)))

      
      



Mean reward for each of the 3 bandits: [0.   0.25 0.  ]
Mean reward for each of the 3 bandits: [23.   21.5  12.25]
Mean reward for each of the 3 bandits: [45.   42.75 25.5 ]
Mean reward for each of the 3 bandits: [72.75 64.   30.5 ]
Mean reward for each of the 3 bandits: [100.5   88.    45.25]
Mean reward for each of the 3 bandits: [125.   108.    52.25]
Mean reward for each of the 3 bandits: [146.   126.    61.75]
Mean reward for each of the 3 bandits: [170.5  150.5   68.75]
Mean reward for each of the 3 bandits: [199.75 170.    78.  ]
Mean reward for each of the 3 bandits: [227.   193.    86.25]
Mean reward for each of the 3 bandits: [251.   210.75  91.5 ]
Mean reward for each of the 3 bandits: [271.75 229.25 103.25]
Mean reward for each of the 3 bandits: [292.25 245.75 111.25]
Mean reward for each of the 3 bandits: [319.75 266.75 116.25]
Mean reward for each of the 3 bandits: [345.5  285.   121.25]
Mean reward for each of the 3 bandits: [372.25 301.5  130.  ]
Mean reward for each of

In [35]:

for a in range(c_bandit.num_bandits):
  print("The agent thinks action " + str(np.argmax(ww[a])+1) + "for bandit "+ str(a+1) +
      "is the most rewarding")
  if np.argmax(ww[a]) == np.argmin(c_bandit.bandits[a]):
        print("...and it was right!")
  else:
        print ("...and it was wrong!")

The agent thinks action 4for bandit 1is the most rewarding
...and it was right!
The agent thinks action 2for bandit 2is the most rewarding
...and it was right!
The agent thinks action 1for bandit 3is the most rewarding
...and it was right!
